[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/dspy/dspy-and-you/Web-Research-Agent-with-Weaviate-and-You.ipynb)

# Combining Web Search with Vector DBs! 

In [2]:
import dspy
from dspy.retrieve.you_rm import YouRM
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate

gpt4 = dspy.OpenAI(model="gpt-4", max_tokens=4000, model_type="chat")

you_rm = YouRM(ydc_api_key=you_api_key, k=1)
weaviate_client = weaviate.Client("http://localhost:8080")
weaviate_rm = WeaviateRM("WeaviateBlogChunk", weaviate_client=weaviate_client)
dspy.settings.configure(lm=gpt4, rm=weaviate_rm)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.4.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [3]:
you_rm("What are cross encoders?").passages[0]

'At this year’s International Conference on Learning Representations (ICLR), we are presenting an unsupervised sentence-pair model we call a trans-encoder (paper, code), which improves on the prior state of the art by up to 5% on sentence similarity benchmarks.\nA tale of two encoders\nToday, there are basically two paradigms for sentence-pair tasks: cross-encoders and bi-encoders. The choice between the two comes down to the standard trade-off between computational efficiency and performance.\nCross-encoder. In a cross-encoder, two sequences are concatenated and sent in one pass to the sentence pair model, which is usually built atop a Transformer-based language model like BERT or RoBERTa. The attention heads of a Transformer can directly model which elements of one sequence correlate with which elements of the other, enabling the computation of an accurate classification/relevance score.'

In [4]:
with dspy.context(rm=weaviate_rm):
    print(dspy.Retrieve(k=5)("What are cross encoders?").passages[0])

[Cross Encoders](#cross-encoders) (collapsing the use of Large Language Models for ranking into this category as well)
1. [Metadata Rankers](#metadata-rankers)
1. [Score Rankers](#score-rankers)

## Cross Encoders
Cross Encoders are one of the most well known ranking models for content-based re-ranking. There is quite a collection of pre-trained cross encoders available on [sentence transformers](https://www.sbert.net/docs/pretrained_cross-encoders.html). We are currently envisioning interfacing cross encoders with Weaviate using the following syntax.


In [5]:
class GenerateAnswer(dspy.Signature):
    """Assess the context and answer the given questions that are predominantly about software usage, process optimization, and troubleshooting. Focus on providing accurate information related to tech or software-related queries."""
    
    blog_context = dspy.InputField(desc="Context from Weaviate Blog Posts.")
    web_context = dspy.InputField(desc="Context from a Web Search.")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="A detailed answer that is supported by the context.")

In [6]:
class RAG(dspy.Module):
    def __init__(self):
        super().__init__()
        
        self.retrieve = dspy.Retrieve(k=1)
        self.you_rm = you_rm
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        blog_contexts = self.retrieve(question).passages
        blog_contexts = "".join(blog_contexts)
        web_contexts = self.you_rm(question).passages
        web_contexts = "".join(web_contexts)
        pred = self.generate_answer(blog_context=blog_contexts, web_context=web_contexts, question=question).answer
        return dspy.Prediction(answer=pred)

In [7]:
RAG()("What are cross encoders?").answer

'Cross encoders are ranking models used for content-based re-ranking. They work by concatenating two sequences and sending them in one pass to the sentence pair model, which is usually built atop a Transformer-based language model like BERT or RoBERTa. The attention heads of a Transformer can directly model which elements of one sequence correlate with which elements of the other, enabling the computation of an accurate classification/relevance score. However, a cross-encoder needs to compute a new encoding for every pair of input sentences, resulting in high computational overhead. Therefore, cross-encoding is often impractical for tasks like information retrieval and clustering, which involve massive pairwise sentence comparisons. Converting pretrained language models into cross-encoders also requires fine-tuning on annotated data.'

In [8]:
gpt4.inspect_history(n=1)





Assess the context and answer the given questions that are predominantly about software usage, process optimization, and troubleshooting. Focus on providing accurate information related to tech or software-related queries.

---

Follow the following format.

Blog Context: Context from Weaviate Blog Posts.

Web Context: Context from a Web Search.

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A detailed answer that is supported by the context.

---

Blog Context: [Cross Encoders](#cross-encoders) (collapsing the use of Large Language Models for ranking into this category as well) 1. [Metadata Rankers](#metadata-rankers) 1. [Score Rankers](#score-rankers) ## Cross Encoders Cross Encoders are one of the most well known ranking models for content-based re-ranking. There is quite a collection of pre-trained cross encoders available on [sentence transformers](https://www.sbert.net/docs/pretrained_cross-encoders.html). We are